# Модель выявления нештатных ситуаций в жизнедеятельности города на основании сообщений жителей, поступающих в режиме реального времени.

### Библиотеки

In [1]:
import numpy as np
import pandas as pd
import datetime as dt
import joblib
import json
import matplotlib.pyplot as plt

import time

from sklearn.cluster import DBSCAN
from sklearn import metrics

Модель на вход принимает параметры сообщения о происшествиях, сдвиг по времени, время жизни данных которые обрабатываются.

На выход модель отдает данные в формате json: о происшествиях, к какому кластеру они относятся и аномалиях.

Для кластеризации данных был выбран алгоритм DBSCAN.

Преимущества:
1. Не требует заранее указывать количество кластеров.
2. Хорошо работает с кластерами произвольной формы.
3. DBSCAN устойчив к выбросам и способен обнаруживать выбросы.

In [6]:
class AlertModel(object):
    
    def __init__(self, data, delta = 3600, lifetime = 60*60*24):
        self.data = data
        
        # внутренний счетчик времени в формате timestamp, нужно проинициализировать минимальной датой-временем в data        
        self.startTime = self.time = data.date.values.min()        
        self.lastTime = data.date.values.max()        
        
        # delta - какой инкремент к внутреннему времени модель делает за один тик:
        self.delta = delta
        
        self.lifetime = lifetime
        self.inc = 0
        
        self.itera = 1
        self.anomaly = False
        self.history = [[[] for j in range(4)] for i in range(7)]
        self.dayOfWeek = self._getDayOfWeek()
        self.dayTime = self._getDayTime()
        self.history[self.dayOfWeek][self.dayTime].append(0)
        
        
    '''
        Функция вычисляет центр кластера точек с географическими координатами. 
        Аргументы : координаты кластера, массив (массив, список списков и т. д.), n - количество точек (широта, долгота) в кластере.
        Возвращает:  геометрический центр кластера 
    '''
    def get_centroid(self, cluster):
        cluster_ary = np.asarray(cluster)
        centroid = cluster_ary.mean(axis = 0)
        return centroid
    
    
    def build_dbscan_model(self, df):
        if len(df) > 1:
            kms_per_rad = 6371.0088
            epsilon = 1.5/kms_per_rad

            # Извлекаем ширину и долготу
            fac_coords = df[['latitude', 'longitude']].values

            start_time = time.time()
            dbsc = (DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine')
                    .fit(np.radians(fac_coords)))
            fac_cluster_labels = dbsc.labels_

            # получаем количеств кластеров
            num_clusters = len(set(dbsc.labels_))
            if num_clusters > 1:
                message = 'Кластеризировано {:,} точек на {:,} кластеров, с сжатием {:.1f}% в {:,.2f} секундах'
                #print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
                #if len(df) > num_clusters:
                #    print('/n')
                #    print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(fac_coords, fac_cluster_labels)))
                #else:
                #    return False
                dbsc_clusters = pd.Series([fac_coords[fac_cluster_labels==n] for n in range(num_clusters)])
    
                # получить центр каждого кластера
                fac_centroids = dbsc_clusters.map(self.get_centroid)

                cent_lats, cent_lons = zip(*fac_centroids)
                centroids_pd = pd.DataFrame({'longitude':cent_lons, 'latitude':cent_lats, 'power': len(df)})
                self.make_json(df, centroids_pd, fac_cluster_labels)
                return centroids_pd
            else:
                return num_clusters
        else:
            return False
        
    
    # Функция создает json файлы для каждой итерации,
    # в котором содержится информация о происшествии, кластере к которму оно принадлежит и аномалиях.
    def make_json(self, df, cen_pd, fac_cluster_labels):
        a = []
        json_data = {}
        b = []
        c = 0
        json_data["incident"] = {}
        json_data["cluster"] = {}
        m = len(df)
        for index, row in df.iterrows():
            if m>=c:
                a.append({'id' : int(row['index']),
                         'date': int(row['date']),
                          'latitude': row['latitude'],
                          'longitude' : row['longitude'],
                          'area' : row['area'],
                          'category':row['category'],
                          'cluster_id': int(fac_cluster_labels[c])
                         })
                c += 1
        json_data["incident"] = a  
        for index, row in cen_pd.iterrows():
            b.append({'id' : int(index),
                     'latitude': cen_pd.latitude.iloc[index] ,
                      'longitude': cen_pd.longitude.iloc[index] ,
                      'power' : int(cen_pd.power.iloc[index]),
                     })
        json_data["cluster"] = b
        
        if self.anomaly != False:
            print(self.anomaly)
            json_data["anomaly"] = self.anomaly
            
        j = json.dumps(json_data,indent = 5, ensure_ascii=False)
        with open("iterations/iteration" + str(self.itera) +".json", "w") as write_file:
            json.dump(json_data, write_file)
        self.itera += 1

        
    def _getDayOfWeek(self):
        return dt.datetime.fromtimestamp(self.time).weekday()
    
    
    def _getDayTime(self):
        return int(dt.datetime.fromtimestamp(self.time).hour // (24/4))
    
    
    #Функция выявления аномалий текущего количество обращений в городские службы по одному из двух критериев: 
    # 1. отклонение от среднего больше чем на дисперсию;
    # 2. выход значений 99 перцентиль по истории фиксации.
    def _detectAnomaly(self):
        data = self.history[self.dayOfWeek][self.dayTime]
        if len(data) < 5:
            self.anomaly = False
            return False  # недостаточно данных для постановки статистического эксперимента
        else:
            #if data[-2] > np.percentile(data[:-2], 99):
            if data[-2] > np.array(data[:-2]).mean() + np.array(data[:-2]).var():
                self.anomaly = {
                    "caption": 'Статистически значимое превышение',
                    "disp": np.array(data[:-2]).var(),
                    "history": data[:-2],
                    "value": data[-2]
                }
                return True
            else:
                self.anomaly = False
                return False
       
    # Функиция итерирует модель на 1 дискрет времени относительно внутреннего времени time
    def iterate(self):
        # Итерирует модель на 1 дискрет времени
        currentIncidentsDf = self.data[(self.data.date >= self.time - self.lifetime) & (self.data.date < self.time + self.delta)]               
        
        if self.dayOfWeek == self._getDayOfWeek() or self.dayTime == self._getDayTime():
            self._detectAnomaly()  # накопились данные для постановки статистического эксперимента
        else:
            self.dayOfWeek = self._getDayOfWeek()
            self.dayTime = self._getDayTime()
            self.history[self.dayOfWeek][self.dayTime].append(0)
              
        incidentsCount = len(self.data[(self.data.date >= self.time) & (self.data.date < self.time + self.delta)])
        self.history[self.dayOfWeek][self.dayTime][-1] += incidentsCount
        
        self.time += self.delta
        self.inc += 1

        return currentIncidentsDf
    
    
    # Функиция итерирует модель до момента timestamp (если не задан - то до текущего момента времени)
    def iterateTo(self, timestamp = int(time.mktime(dt.datetime.now().timetuple()))):
        i = 0
        while self.time < min(timestamp, self.lastTime):
            res = self.iterate()
            model_dbscan = self.build_dbscan_model(res)
            if i % 100 == 0:
                print('\n', dt.datetime.fromtimestamp(self.time), len(res))
            #else:
             #   print(' ', end = '')
            i += 1
        return True # если итерации прошли успешно возвращает True, иначе False
    
    
    

In [3]:
# Данные для анализа
df = pd.read_csv('data/Датасет4_Пожары.xlsx - mess_АпкБг1016Пожары.csv')
df.reset_index(level=0, inplace=True)

df.columns = ['index', 'registration_time', 'category', 'id_addres', 'id_building', 'latitude', 'longitude', 'area']
data = df[['index', 'registration_time', 'latitude', 'longitude', 'category', 'area']]
data = data[data['latitude'] != 0]
data = data[data['longitude'] < 45]
# Преобразовать в unix timestamp:
data['registration_time'] = pd.to_datetime(df['registration_time'])
# data['date'] = pd.to_datetime(data['registration_time']).astype(int)/ 10**9
data['date'] = [int((dt - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')) for dt in pd.to_datetime(data['registration_time']).values]

<ipython-input-3-7f34ede75e48>:12: DeprecationWarning: parsing timezone aware datetimes is deprecated; this will raise an error in the future
  data['date'] = [int((dt - np.datetime64('1970-01-01T00:00:00Z')) / np.timedelta64(1, 's')) for dt in pd.to_datetime(data['registration_time']).values]


In [7]:
model = AlertModel(data)

model.iterateTo()


 2019-01-01 04:06:00 12

 2019-01-05 08:06:00 31

 2019-01-09 12:06:00 31

 2019-01-13 16:06:00 38

 2019-01-17 20:06:00 59

 2019-01-22 00:06:00 55

 2019-01-26 04:06:00 60

 2019-01-30 08:06:00 60

 2019-02-03 12:06:00 59

 2019-02-07 16:06:00 62

 2019-02-11 20:06:00 43

 2019-02-16 00:06:00 40

 2019-02-20 04:06:00 35

 2019-02-24 08:06:00 35

 2019-02-28 12:06:00 39

 2019-03-04 16:06:00 75

 2019-03-08 20:06:00 48

 2019-03-13 00:06:00 31

 2019-03-17 04:06:00 48

 2019-03-21 08:06:00 13

 2019-03-25 12:06:00 58

 2019-03-29 16:06:00 50

 2019-04-02 20:06:00 44

 2019-04-07 00:06:00 91

 2019-04-11 04:06:00 50

 2019-04-15 08:06:00 85

 2019-04-19 12:06:00 109

 2019-04-23 16:06:00 70

 2019-04-27 20:06:00 117

 2019-05-02 00:06:00 98
{'caption': 'Статистически значимое превышение', 'disp': 29.95918367346939, 'history': [33, 40, 40, 35, 30, 23, 34], 'value': 75}
{'caption': 'Статистически значимое превышение', 'disp': 29.95918367346939, 'history': [33, 40, 40, 35, 30, 23, 34], '

True